In [14]:
import pandas as pd
import numpy as np
import miceforest as mf
from math import nan
# Note: properly installing lightgbm allows you to run miceforest. If you have an M1 mac, please see:
# https://towardsdatascience.com/install-xgboost-and-lightgbm-on-apple-m1-macs-cb75180a2dda

In [26]:
student_survey = pd.read_csv('data/36423-0002-Data.tsv',sep='\t')

### Math Enrollment

S2MSPR12: S2 D13 Teenager taking math class(es) in spring 2012
[Are you currently/Were you] taking a math course [during the spring term of 2012?]
1=Yes
0=No

S1MFALL09: S1 C03 9th grader is taking a math course in the fall 2009 term

In [27]:
student_survey = student_survey[(student_survey['S2MSPR12'] == 1) & (student_survey['S1MFALL09'] == 1)]
len(student_survey)

14575

### Sample size rounded to the nearest ten in accordance to NCES regulation.
Throughout their paper.

### Low teacher support 

S2MTCHTREAT: S2 D18A Teen's spring 2012 math teacher treats some kids better than others
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[treats/treated] some kids better than other kids.
1=Strongly agree
2=Agree

S2MTCHINTRST: S2 D18B Teen's spring 2012 math teacher makes math interesting
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[makes/made] math interesting.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree


S2MTCHEASY: S2 D18C Teen's spring 2012 math teacher makes math easy to understand
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[makes/made] math easy to understand.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

S2MTCHTHINK: S2 D18D Teen's spring 2012 math teacher wants students to think, not
memorize
How much do you agree or disagree with the following statements about your teacher for [math course
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[wants/wanted] students to think, not just memorize things.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

S2MTCHGIVEUP: S2 D18E Teen's spring 2012 math teacher doesn't let students give up
title]? Remember, none of your teachers or your principal will see any of the answers you provide.
Your teacher...
[doesn't/didn't] let people give up when the work [gets/got] hard.
1=Strongly agree
2=Agree
3=Disagree
4=Strongly disagree

### Ability self-concepts

S1MTESTS: S1 C08A 9th grader confident can do excellent job on fall 2009 math tests
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are confident that you can do an excellent job on tests in this course
Strongly agree
Agree

S1MTEXTBOOK: S1 C08B 9th grader certain can understand fall 2009 math textbook
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are certain that you can understand the most difficult material presented in the textbook used
in this course
Strongly agree
Agree
Disagree
Strongly disagree

S1MSKILLS: S1 C08C 9th grader certain can master skills in fall 2009 math course
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are certain that you can master the skills being taught in this course
Strongly agree
Agree
Disagree
Strongly disagree

S1MASSEXCL: S1 C08D 9th grader confident can do excellent job on fall 2009 math
assignments
How much do you agree or disagree with the following statements about your [fall 2009 math] course?
You are confident that you can do an excellent job on assignments in this course
Strongly agree
Agree
Disagree
Strongly disagree

### Parental support

P1MUSEUM: P1 E07A Went to science or engineering museum with 9th grader in last year
P2MUSEUM: P2 B10A Visited science-related destination together in last year
Value Label Unweighted
Frequency
%
0 No 7195 30.6 %
1 Yes 8253 35.1 %
Missing Data
-9 Missing 1340 5.7 %
-8 Unit non-response 6715 28.6 %

Value Label Unweighted
Frequency
%
0 No 4837 20.6 %
1 Yes 3248 13.8 %
Missing Data
-9 Missing 63 0.3 %
-8 Unit non-response 2603 11.1 %
-6 Component not applicable 12279 52.2 %
-4 Item not administered: abbreviated interview 473 2.0 %

P1COMPUTER: P1 E07B Worked or played on computer with 9th grader in last year
P2COMPUTER: P2 B10B Worked or played on computer with teenager in last year

P1FIXED: P1 E07C Built or fixed something with 9th grader in last year
P2FIXED: P2 B10C Built or fixed something with teenager in last year

P1LIBRARY: P1 E07G Visited a library with 9th grader in last year
P2LIBRARY: P2 B10F Visited a library with teenager in last year

P1STEMDISC: P1 E07F Discussed STEM program or article with 9th grader in last year
P2STEMDISC: P2 B10E Discussed STEM program or article with teenager in last year

### Math Acheivement Score

X2TXMSCR: X2 Mathematics IRT-estimated number right score (of ## first follow-up items)
Based upon 20,594 valid cases out of 23,503 total cases.
• Mean: 67.2219
• Minimum: 25.0057
• Maximum: 115.1000
• Standard Deviation: 19.2183

X2X1TXMSCR: X2 Mathematics IRT-estimated number right score at time of base year (of 118
first follow-up items)

X3THIMATH9: X3 Highest level mathematics course taken - ninth grade
13 AP/IB Calculus 0 0.0 %
Missing Data
-9 Missing 770 3.3 %
-8 Unit non-response

In [28]:
all_columns = [
    'S2MSPR12',
    'S1MFALL09',
    'S2MTCHTREAT', 
    'S2MTCHINTRST',
    'S2MTCHEASY',
    'S2MTCHTHINK',
    'S2MTCHGIVEUP',
    'S1MTESTS',
    'S1MTEXTBOOK',
    'S1MSKILLS',
    'S1MASSEXCL',
    'P1MUSEUM',
    'P2MUSEUM',
    'P1COMPUTER',
    'P2COMPUTER',
    'P1FIXED',
    'P2FIXED',
    'P1LIBRARY',
    'P2LIBRARY',
    'P1STEMDISC',
    'P2STEMDISC',
    'X2TXMSCR',
    'X2X1TXMSCR',
    'X1SEX',
    'X1RACE',
    'X1SES_U',
    'X3THIMATH9'
]

low_teacher_support = [
    ('S2MTCHTREAT','treats some kids better'),
    ('S2MTCHINTRST','makes math interesting'),
    ('S2MTCHEASY','makes math easy to understand'),
    ('S2MTCHTHINK','wants students to think'),
    ('S2MTCHGIVEUP','doesnt let students give up')
]

ability_self_concept = [
    ('S1MTESTS','confident can do excellent job on test'),
    ('S1MTEXTBOOK','certain can understand math textbook'),
    ('S1MSKILLS','certain can master math skills'),
    ('S1MASSEXCL','confident can do excellent job on assignments')
]

parental_support = [
    ('P1MUSEUM','went to science or engineering museum'),
    ('P2MUSEUM','went to science or engineering museum'),
    ('P1COMPUTER','worked or played on computer'),
    ('P2COMPUTER','worked or played on computer'),
    ('P1FIXED','built or fixed something'),
    ('P2FIXED','built or fixed something'),
    ('P1LIBRARY','visited a library'),
    ('P2LIBRARY','visited a library'),
    ('P1STEMDISC','discussed STEM program or article'),
    ('P2STEMDISC','discussed STEM program or article'),]

covariates = [
    'sex',
    'race',
    'SES',
    'base_year_score',
    'highest_level_math'
]

math_acheivement_score = [('X2TXMSCR', 'score')]

Students’ demographic information including their gender, race/ethnicity, 9 grade math achievement using the IRT-estimated
score (i.e., a criterion-referenced measure of achievement on algebraic reasoning assessment which was similarly constructed and administered as the assessment in 11th grade), and socioeconomic status (i.e., a composite measure of parents’ education, occupation, and family income) collected in 9th grade were included in the analyses as covariates. Also, students’ highest-level math course taken in 9th grade (1 = Basic math, 13 = AP/IB calculus) from the high school transcript was included.

In [29]:
student_survey = student_survey[all_columns]

student_survey['sex'] = student_survey['X1SEX']
student_survey.loc[student_survey["sex"] == 1, "sex"] = 0
student_survey.loc[student_survey["sex"] == 2, "sex"] = 1
student_survey['sex'] = student_survey['sex'].astype(np.int8)

student_survey['race'] = student_survey['X1RACE']
student_survey.loc[student_survey["race"] == 8, "race"] = 10
student_survey.loc[student_survey["race"] == 3, "race"] = 11
student_survey.loc[student_survey["race"] == 4, "race"] = 12
student_survey.loc[student_survey["race"] == 5, "race"] = 12
student_survey.loc[student_survey["race"] == 2, "race"] = 13
student_survey.loc[student_survey["race"] == 6, "race"] = 14
student_survey.loc[student_survey["race"] == 7, "race"] = 15
student_survey.loc[student_survey["race"] == 1, "race"] = 15
student_survey['race'] = student_survey['race'] - 10
student_survey['race'] = student_survey['race'].astype(np.int8)

student_survey['SES'] = pd.qcut(student_survey['X1SES_U'],4,labels=np.arange(4) + 1)
student_survey['SES'] = student_survey['SES'].astype(np.int8)

In [30]:
student_survey = student_survey.apply(pd.to_numeric, errors = 'coerce')
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,P2STEMDISC,X2TXMSCR,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,sex,race,SES
0,1,1,2,2,2,2,3,1,2,2,...,1,99.1403,69.4994,1,8,1.6907,6,0,0,4
1,1,1,3,3,3,2,2,2,2,2,...,0,72.4904,49.4710,2,8,-0.3923,2,1,0,2
2,1,1,3,1,1,2,1,1,3,2,...,1,75.4243,77.3584,2,3,1.1271,5,1,1,4
6,1,1,2,4,4,2,3,2,2,2,...,-6,60.0290,52.7841,2,8,-0.4774,3,1,0,1
9,1,1,2,3,2,2,3,1,1,1,...,1,86.8286,76.6330,2,8,0.1081,4,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3,1,1,1,1,2,2,2,...,-6,85.8476,70.7533,1,8,1.2713,-9,0,0,4
23499,1,1,3,3,2,1,1,2,1,2,...,-4,60.8373,46.4682,2,5,-1.3350,3,1,2,1
23500,1,1,1,1,1,1,1,2,3,2,...,0,60.9564,53.2958,2,8,-0.0031,4,1,0,2
23501,1,1,2,4,4,3,4,2,2,2,...,1,65.1187,72.6301,1,8,0.7236,4,0,0,4


### Imputation
The authors specify that they use ``multiple imputation'' procedures to impute missing data. Because they do not specify which procedure they use (beyond that they use the STATA package), we do best practice work here (in python) and use an MI library based on LightGBM and the MICE algorithm.

In [31]:
student_survey[student_survey < 0] = nan
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,P2STEMDISC,X2TXMSCR,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,sex,race,SES
0,1,1,2.0,2.0,2.0,2.0,3.0,1.0,2.0,2.0,...,1.0,99.1403,69.4994,1,8,1.6907,6.0,0,0,4
1,1,1,3.0,3.0,3.0,2.0,2.0,2.0,2.0,2.0,...,0.0,72.4904,49.4710,2,8,NaN,2.0,1,0,2
2,1,1,3.0,1.0,1.0,2.0,1.0,1.0,3.0,2.0,...,1.0,75.4243,77.3584,2,3,1.1271,5.0,1,1,4
6,1,1,2.0,4.0,4.0,2.0,3.0,2.0,2.0,2.0,...,NaN,60.0290,52.7841,2,8,NaN,3.0,1,0,1
9,1,1,2.0,3.0,2.0,2.0,3.0,1.0,1.0,1.0,...,1.0,86.8286,76.6330,2,8,0.1081,4.0,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,...,NaN,85.8476,70.7533,1,8,1.2713,NaN,0,0,4
23499,1,1,3.0,3.0,2.0,1.0,1.0,2.0,1.0,2.0,...,NaN,60.8373,46.4682,2,5,NaN,3.0,1,2,1
23500,1,1,1.0,1.0,1.0,1.0,1.0,2.0,3.0,2.0,...,0.0,60.9564,53.2958,2,8,NaN,4.0,1,0,2
23501,1,1,2.0,4.0,4.0,3.0,4.0,2.0,2.0,2.0,...,1.0,65.1187,72.6301,1,8,0.7236,4.0,0,0,4


In [ ]:
# Using pip
# ! pip install miceforest --no-cache-dir

In [32]:
# Create kernel. 
kds = mf.ImputationKernel(
  student_survey,
  datasets=1,
  save_all_iterations=False,
  random_state=42
)

# Run the MICE algorithm for 2 iterations
kds.mice(2)



In [35]:
completed_dataset = kds.complete_data(dataset=0, inplace=False)
student_survey.describe() - completed_dataset.describe()

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,P2STEMDISC,X2TXMSCR,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,sex,race,SES
count,0.0,0.0,-243.000000,-243.000000,-252.000000,-257.000000,-271.00000,-67.000000,-98.000000,-114.000000,...,-8913.000000,0.0,0.0,0.0,0.0,-6624.000000,-1057.000000,0.0,0.0,0.0
mean,0.0,0.0,-0.001860,0.000527,0.002806,0.001508,0.00051,-0.000347,-0.000346,-0.000298,...,0.008227,0.0,0.0,0.0,0.0,0.306638,0.000528,0.0,0.0,0.0
std,0.0,0.0,0.000097,0.001265,0.001387,0.001619,0.00180,0.000759,0.000898,0.000238,...,-0.003888,0.0,0.0,0.0,0.0,0.019668,-0.002706,0.0,0.0,0.0
min,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
25%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.230550,0.000000,0.0,0.0,0.0
50%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.518300,0.000000,0.0,0.0,0.0
75%,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.384050,0.000000,0.0,0.0,0.0
max,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [21]:
#student_survey = student_survey[(student_survey[all_columns] >= 0).all(axis=1)]
student_survey

,S2MSPR12,S1MFALL09,S2MTCHTREAT,S2MTCHINTRST,S2MTCHEASY,S2MTCHTHINK,S2MTCHGIVEUP,S1MTESTS,S1MTEXTBOOK,S1MSKILLS,...,P2STEMDISC,X2TXMSCR,X2X1TXMSCR,X1SEX,X1RACE,X1SES_U,X3THIMATH9,sex,race,SES
0,1,1,2,2,2,2,3,1,2,2,...,1,99.1403,69.4994,1,8,1.6907,6,0,0,4
1,1,1,3,3,3,2,2,2,2,2,...,0,72.4904,49.4710,2,8,-0.3923,2,1,0,2
2,1,1,3,1,1,2,1,1,3,2,...,1,75.4243,77.3584,2,3,1.1271,5,1,1,4
6,1,1,2,4,4,2,3,2,2,2,...,-6,60.0290,52.7841,2,8,-0.4774,3,1,0,2
9,1,1,2,3,2,2,3,1,1,1,...,1,86.8286,76.6330,2,8,0.1081,4,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23497,1,1,3,1,1,1,1,2,2,2,...,-6,85.8476,70.7533,1,8,1.2713,-9,0,0,4
23499,1,1,3,3,2,1,1,2,1,2,...,-4,60.8373,46.4682,2,5,-1.3350,3,1,2,1
23500,1,1,1,1,1,1,1,2,3,2,...,0,60.9564,53.2958,2,8,-0.0031,4,1,0,3
23501,1,1,2,4,4,3,4,2,2,2,...,1,65.1187,72.6301,1,8,0.7236,4,0,0,4
